In [ ]:
import os
import json

import numpy as np
import pandas as pd
import keras
from keras import layers
from keras.applications.densenet import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

You can find the data for this competition here: https://www.kaggle.com/c/iwildcam-2019-fgvc6/data 

In [ ]:
os.listdir('../input')

In [ ]:
# The data, split between train and test sets:
x_train = np.load('../input/reducing-image-sizes-to-32x32/X_train.npy')
x_test = np.load('../input/reducing-image-sizes-to-32x32/X_test.npy')
y_train = np.load('../input/reducing-image-sizes-to-32x32/y_train.npy')

y_train.shape


In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255



In [ ]:
class Metrics(Callback):
    def on_train_begin(self,logs = {}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
        
    def on_epoch_end(self,epoch, logs = {}):
        X_val, y_val = self.validation_data[:2]
        y_pred = self.model.predict(X_val)
        
        y_pred_cat = keras.utils.to_categorical(
        
            y_pred.argmax(axis = 1),
            
            num_classes = 14
        )
        
        
        _val_f1 = f1_score(y_val, y_pred_cat , average = 'macro')
        _val_recall = recall_score(y_val, y_pred_cat, average = 'macro')
        _val_precision = precision_score(y_val, y_pred_cat, average = 'macro')
        
        
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        
        print((f"validation f1-macro : {_val_f1 : .4f}" f"validation recall : {_val_recall : .4f}" f"validation precision : {_val_precision : .4f}"))
        
        
f1_metrics = Metrics()

In [ ]:
densenet = DenseNet121(weights = '../input/densenet-keras/DenseNet-BC-121-32-no-top.h5', include_top= False, input_shape = (32, 32, 3))

In [ ]:
model = Sequential()
model.add(densenet)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(14, activation="softmax"))

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

checkpoint = ModelCheckpoint('model.h5', monitor = 'val_acc', verbose = 1, save_best_only = True, save_weights_only = False, mode = 'auto')


history = model.fit(x = x_train, y= y_train, batch_size = 64, epochs = 6, callbacks = [checkpoint, f1_metrics], validation_split = 0.1)

In [ ]:
model.load_weights('model.h5')
y_test = model.predict(x_test)

submission_df = pd.read_csv ('../input/iwildcam-2019-fgvc6/sample_submission.csv')
submission_df['Predicted'] = y_test.argmax(axis = 1)

print(submission_df.shape)
submission_df.head()